[DeepPavlov](http://docs.deeppavlov.ai/en/0.7.0/index.html) - это библиотека для решения различных NLP-задач от МФТИ, у них есть свой синтаксический анализатор

[документация](http://docs.deeppavlov.ai/en/0.7.0/features/models/syntaxparser.html)

модель выдает результат в CONLL-U формате и обучена на   [UD-трибанках](http://universaldependencies.org/format.html)

In [ ]:
# установка (может быть долгой)
!pip3 install deeppavlov
!pip3 install russian-tagsets
!python3 -m deeppavlov install syntax_ru_syntagrus_bert

In [ ]:
import deeppavlov

In [ ]:
# импортим модель (здесь тоже долго..)
from deeppavlov import build_model, configs 
dpavlov_model = build_model("ru_syntagrus_joint_parsing", download=True)

In [ ]:
sentences = ["Собянин открыл новый парк и детскую площадку"]
#model['main'].to_output_string = False
#model['main'].output_format = 'dict'

for parse in dpavlov_model(sentences): # аргумент должен быть списком
    print(parse)

### шаг3 текст

создадим какой-нибудь небольшой текст, на котором будем тестить модель:

    1	Собянин	_	NOUN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing|fPOS=NOUN++	2	nsubj	_	_
    2	открыл	_	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act|fPOS=VERB++	0	ROOT	_	_
    3	новый	_	ADJ	_	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing|fPOS=ADJ++	4	amod	_	_
    4	парк	_	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing|fPOS=NOUN++	2	dobj	_	_
    5	и	_	CONJ	_	fPOS=CONJ++	4	cc	_	_
    6	детскую	_	ADJ	_	Case=Acc|Degree=Pos|Gender=Fem|Number=Sing|fPOS=ADJ++	7	amod	_	_
    7	площадку	_	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing|fPOS=NOUN++	4	conj	_	_
    8	.	_	PUNCT	.	fPOS=PUNCT++.	2	punct	_	_

## визуализация

В nltk есть DependencyGraph, который умеет рисовать деревья (и ещё многое другое). Для того, чтобы визуализация работала корректно, ему нужна зависимость: graphviz.

In [ ]:
# ! pip3 install graphviz
# ! pip3 install pydot 
# ! brew install graphviz

Бибилиотеку grapghviz нужно поставить не только через pip, но и добавить в систему ([см этот тред](https://stackoverflow.com/questions/35064304/runtimeerror-make-sure-the-graphviz-executables-are-on-your-systems-path-aft))

Если возникает ошибка, попробуйте следующие команды:
* (Win) запустите в ячейке код  
```import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/' ```

* (Mac) в терминале: ```brew install graphviz ``` (проверьте, что у Вас стоит [homebrew](https://brew.sh/))

* (Linux) в терминале: ```sudo apt-get install graphviz```

In [ ]:
from nltk import DependencyGraph, Tree 
# вызываем классы, которые нарисуют нам деревья зависимостей

Для построения дерева нам нужно превратить файл в conllu-формате в список
<br> 
Еще нужно сделать тег ROOT в верхнем регистре, иначе он не находится

Создадим функцию, которую попробуем на результате UDPipe и на результате DeepPavlov

In [ ]:
def conllu_to_list(parser_result): 
# аргумент - это conllu-файл, который получили в результате синтаксического анализа 
    sents = []
    for sent in parser_result.split('\n\n'):
        # убираем коменты
        sent = '\n'.join([line for line in sent.split('\n') if not line.startswith('#')])
        # заменяем регистр для root
        sent = sent.replace('\troot\t', '\tROOT\t')
        sents.append(sent)
        return sents

In [ ]:
#теперь - применяем к модели deeppavlov

dp = conllu_to_list(dpavlov_model(sentences)[0])

In [ ]:
dp[0]

In [ ]:
# нарисуем граф для DeepPavlov
dp_graph = DependencyGraph(tree_str=dp[0])
dp_graph

In [ ]:
# дерево для DeepPavlov
dp_tree = dp_graph.tree()
print(dp_tree.pretty_print())

In [ ]:
# посмотрим как можно разбить предложение
list(dp_graph.triples())

### Тройки глагол-объект-субьект:
Предположим, нам нужно вытащить только ту тройку, которая расскажет о предикате (сказуемом), субъекте (подлежащем) и объекте (дополнении)

In [ ]:
def get_sov(sent): # зададим функцию, которая будет вытаскивать нужную тройку
    dp_graph = DependencyGraph(tree_str=dp[0]) 
    sov = {} # пустой словарь, будем в него складывать
    for triple in graph.triples(): # для каждого триплета из всех
        if triple:
            if triple[0][1] == 'VERB': # если тег первого элемента - VERB
                sov[triple[0][0]] = {'subj':'','obj':''}
    for triple in graph.triples():
        if triple:
            if triple[1] == 'nsubj':
                if triple[0][1] == 'VERB':
                    sov[triple[0][0]]['subj']  = triple[2][0]
            if triple[1] == 'obj':
                if triple[0][1] == 'VERB':
                    sov[triple[0][0]]['obj'] = triple[2][0]
    return sov

sov = get_sov(sent)
print(sov)

улучшим функцию, теперь она находит однородные дополнения *(парк и площадку)*

In [ ]:
from collections import defaultdict as dd


def get_sov(sent):
    graph = DependencyGraph(tree_str=dp[0])

    subjects = dd(lambda : {"subject": "", "verb": "", "objects": []})
    verbs = dd(lambda : {"subject": "", "verb": "", "objects": []})
    
    for triple in graph.triples():
        if triple:
            
            if triple[1] == 'conj':
                subjects[triple[0][0]]["objects"].append(triple[2][0])
                
            if triple[1] == 'nsubj':
                if triple[0][1] == 'VERB':
                    verbs[triple[0][0]]["subject"] = triple[2][0]
            if triple[1] == 'obj':
                if triple[0][1] == 'VERB':
                    subjects[triple[2][0]]["verb"] = triple[0][0]
                    subjects[triple[2][0]]["objects"].append(triple[2][0])

    
    sovs = []

    print(subjects, verbs)
    
    for v in subjects.values():
        for obj in v["objects"]:
            sovs.append((verbs[v["verb"]]["subject"], v["verb"], obj))
    return sovs

sov = get_sov(sent)
print("\n",sov[-2:])